In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pprint import pprint

DATABASE='hr'
TABLE = 'employees'


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Sample PySpark Application") \
    .getOrCreate()
'''
.config("spark.hadoop.fs.s3a.access.key", os.environ['AWS_ACCESS_KEY_ID'])\
.config("spark.hadoop.fs.s3a.secret.key", os.environ['AWS_SECRET_ACCESS_KEY'])\
.config("spark.hadoop.fs.s3a.endpoint", os.environ['AWS_ENDPOINT_URL_S3'])\
'''

23/12/05 19:16:10 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
data = [("James","","Smith","36636","M",3001),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])

In [6]:
df = spark.createDataFrame(data=data, schema=schema)
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



In [7]:
#spark.sql("ALTER TABLE hr.employees SET TBLPROPERTIES('external'='false');")
#spark.sql(f"describe  {DATABASE}.{TABLE} ;")

In [8]:
spark.sql(F'Drop table IF EXISTS {DATABASE}.{TABLE} PURGE;')
spark.sql(F'Drop DATABASE IF EXISTS {DATABASE} cascade;')

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


DataFrame[]

In [9]:
spark.sql(F'CREATE DATABASE IF NOT EXISTS {DATABASE};')
df.writeTo(f"{DATABASE}.{TABLE}").createOrReplace()

In [10]:
spark.catalog.listDatabases()

[Database(name='default', catalog='lakehouse', description='Default Hive database', locationUri='s3://datalake/published/warehouse'),
 Database(name='hr', catalog='lakehouse', description=None, locationUri='s3://datalake/published/warehouse/hr.db'),
 Database(name='nyc', catalog='lakehouse', description=None, locationUri='s3://datalake/published/warehouse/nyc.db')]

In [11]:
spark.catalog.listTables(DATABASE)

[Table(name='employees', catalog='lakehouse', namespace=['hr'], description=None, tableType='MANAGED', isTemporary=False)]

In [12]:
spark.stop()